Question 1a: Cryptography.
File for question 1a – crypto.txt
i.
Create a Google Colab notebook titled "Cryptography" and install the necessary cryptographic modules.
ii.
Generate an RSA key pair (public and private keys) with a key size of 2048 bits.
iii.
Encrypt the contents of the crypto.txt file using AES encryption with a randomly generated AES key (16 bytes).
iv.
Encrypt the randomly generated AES key using RSA encryption with the generated public key. 
v.
Generate a digital signature for the encrypted data using the RSA private key.
vi.
Save the encrypted data, encrypted AES key, and the digital signature into separate files and name the files as
"crypto_encrypted.txt", "aes_key_encrypted.bin", and "signature.bin" respectively.

In [1]:
pip install pycryptodome

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Generate an RSA key pair (public and private keys) with a key size of 2048 bits.
from Crypto.PublicKey import RSA

key = RSA.generate(2048) #generate the key pair
privateKey = key.export_key() #export private key to privateKey variable (the private key contains both private and public key components)
publicKey = key.publickey().export_key() #export just the public key to the publicKey variable

with open("privateKey.pem", "wb") as privateKeyFile: #create a file named privateKey.pem, open it, and point to it with a reference privateKeyFile
    privateKeyFile.write(privateKey) #write the private key to the file

with open("publicKey.pem", "wb") as publicKeyFile: #create a file named publicKey.pem, open it and point to it with a reference publicKeyFile
    publicKeyFile.write(publicKey) #write the public key to the file

In [3]:
#Encrypt the contents of the crypto.txt file using AES encryption with a randomly generated AES key (16 bytes)
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

key = get_random_bytes(16) #create a key using 16 random bytes

with open("crypto.txt", "rb") as fileToEncrypt:
    cipher = AES.new(key, AES.MODE_GCM) #create AES Cipher object used for encryption
    #NOTE: I chose GCM as it seems to be the most secure mode for AES
    
    cipherText, tag = cipher.encrypt_and_digest(fileToEncrypt.read()) #Encrypt the file using the cipher, also creates an authenticity tag to save for decryption
    nonce = cipher.nonce #stores the nonce, used for decryption
    
    with open("crypto_encrypted.txt", "wb") as cryptoEncrypted:
        cryptoEncrypted.write(cipherText)

In [4]:
#Encrypt the randomly generated AES key using RSA encryption with the generated public key.
from Crypto.Cipher import PKCS1_OAEP

with open("publicKey.pem", "rb") as pubKeyFile:
    publicKey = RSA.import_key(pubKeyFile.read()) #import RSA key from publicKey.pem

rsaCipher = PKCS1_OAEP.new(publicKey) #create a cipher instance using PKCS1_OAEP padding
encryptedAesKey = rsaCipher.encrypt(key) #encrypt the AES key using the cipher

with open("aes_key_encrypted.bin", "wb") as aesKeyFile:
    aesKeyFile.write(encryptedAesKey)

In [5]:
#Generate a digital signature for the encrypted data using the RSA private key. [2]
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256

with open("privateKey.pem", "rb") as privKeyFile:
    key = RSA.import_key(privKeyFile.read()) #import the RSA private key

with open("crypto_encrypted.txt", "rb") as encryptedData:
    h = SHA256.new(encryptedData.read()) #hash the encrypted data using SHA256

signature = pkcs1_15.new(key).sign(h) #sign the data using the RSA private key

with open("signature.bin", "wb") as signatureFile:
    signatureFile.write(signature)

